In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jan_dict_JSON = json.load(json_file)

In [10]:
weekend_Jan_list = [1, 7, 8, 14, 15, 21, 22, 28, 29]

In [11]:
link_day_minute_Jan_list = []
for link_idx in range(24):
    for day in weekend_Jan_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jan_list.append(link_day_minute_Jan_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jan_list)

25920

In [13]:
x = np.matrix(link_day_minute_Jan_list)
x = np.matrix.reshape(x, 24, 1080)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
x[:,:2]

matrix([[ 3903.22863242,  4242.98320757],
        [ 4012.46701366,  3120.52796148],
        [ 5691.68393741,  5336.67776288],
        [ 4527.89931295,  4509.97567332],
        [ 5695.22658189,  5736.27386271],
        [ 6265.37546155,  5707.98847921],
        [ 5270.66850029,  4963.43625806],
        [ 4389.56113904,  4272.73140244],
        [ 4432.80777906,  4423.61110269],
        [ 5298.76117144,  5061.34716028],
        [ 5102.15470676,  6282.53713048],
        [ 3826.67506755,  4639.91567803],
        [ 4617.11897802,  4064.12260807],
        [ 2773.56110633,  3930.01711357],
        [ 4293.40930334,  1919.52084127],
        [ 3023.9330937 ,  3432.09087348],
        [ 4373.65598092,  3428.71656921],
        [ 4670.51453081,  4549.3721486 ],
        [ 3285.48740622,  4012.23831437],
        [ 4785.1938381 ,  3905.4084889 ],
        [ 1934.53854541,  2398.9583872 ],
        [ 2222.62277852,  2330.74909154],
        [ 4132.17932147,  3661.52117374],
        [ 5120.70575521,  4812.459

In [15]:
np.size(x, 0), np.size(x, 1)

(24, 961)

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [1e-01, 1e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.11197830e+05  0.00000000e+00  3.63e+03 1.59e+02  9.80e+05     0s
   1   5.92108293e+05 -7.32846407e+05  9.69e+02 2.66e+02  2.82e+05     0s
   2   6.40356744e+05 -8.04719986e+05  2.75e+01 7.03e+00  3.39e+04     0s
   3   1.08739611e+05 -2.36505545e+05  1.43e+00 1.14e-13  6.3

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jan_weekend.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[1.6244371677484336e-09,
 5972.48909092056,
 1.7998043662220864e-09,
 1.7593478438705525,
 1106.880790173908,
 6.971193377463536e-06,
 5.03978744326971e-07,
 0.008306981706679374,
 9.771369683228287e-09,
 3985.4096442090986,
 9.771369871367301e-09,
 9.77136968352533e-09,
 9.757692538335755e-09,
 9.813647480688657e-09,
 4668.57098468178,
 1037.888890955132,
 2.728743487446838,
 1.8723088288516945,
 1735.0270682127718,
 6788.032513141763,
 6.973255407661406e-06,
 1528.0016978783171,
 2663.4073168618884,
 405.31896006288537,
 1882.7045934584412,
 864.0351148423714,
 3.9966917553931275e-06,
 3376.3279427147177,
 2.1372394228458234,
 2.755301242315767,
 454.57121588031566,
 4.168597747614285,
 1835.3121666124164,
 3.956630878524318e-06,
 3.9586404190172234e-06,
 1719.9676465159603,
 531.6290999030291,
 0.002199244233466402,
 1896.7014352015358,
 401.9818526709239,
 193.71616176008243,
 108.1033745669775,
 5.021103428580138e-06,
 5.021103453229481e-06,
 5785.387427865429,
 866.268410878145,
